In [ ]:
#add_item  https://isvvrv2smqqpqgrobppq7t4hkq0midra.lambda-url.us-east-1.on.aws/
import json
import boto3

client = boto3.client('dynamodb')

def lambda_handler(event, context):
    # TODO implement
    data = json.loads(event['body'])
    print(data)
    table = data['table']
    username = data['user_name']
    passkey = data['passkey']
    nickname =data['nickname']
    if data.__contains__('devnum'):
        num= int(data['devnum'])
        st = ""
        for i in range(num):
            st=st+str(data[('devcid'+str(i))])
        devcid = st
    else:  
        if data.__contains__('devcid'):
            devcid = data['devcid']

    id =  data['id']
    if id == '' :
        id = username
    res = client.put_item(
        TableName=table,
        Item={
            'id':{'S':id},
            'username': {'S':username},
            'passkey': {'S':passkey},
            'devcid': {'S':devcid},
            'nickname':{'S':nickname},
            'avatar':{'N':'0'},
            'avatar_file_url':{'S':''},
            'imagedata':{'B':''},
            'mobile':{'S':''},
            
        },
        ConditionExpression='attribute_not_exists(id)',
    )
    return {
        'statusCode': 200,
        'body': json.dumps('OK')
    }


In [ ]:
#update_item  https://7pakgjr5vudpkdoroqhuheuoky0oikfv.lambda-url.us-east-1.on.aws/
import json
import boto3

client = boto3.client('dynamodb')

def lambda_handler(event, context):
    # TODO implement
    update_str = 'SET '
    update_val = {}
    print(event)
    data = json.loads(event['body'])
    update_str +=' oldpasskey=:pass'
    update_val[':pass'] = {'S':data['passkey']}
    
    if data.__contains__('devnum'):
        update_str +=', devcid=:val2'
        num= int(data['devnum'])
        st = ""
        for i in range(num):
            st=st+str(data[('devcid'+str(i))])
            print(st)
        update_val[':val2'] = {'S':st}  
    else:  
        if data.__contains__('devcid'):
            update_str +=', devcid=:val2'
            update_val[':val2'] = {'S':data['devcid']}
    if data.__contains__('imagedata'):
        update_str +=', imagedata=:val3'
        update_val[':val3'] = {'B':data['imagedata']}
    if data.__contains__('nickname'):
        update_str +=', nickname=:val4'
        update_val[':val4'] = {'S':data['nickname']}
    if data.__contains__('newpasskey'):
        update_str +=', passkey=:new'
        update_val[':new'] = {'S':data['newpasskey']}
    if data.__contains__('avatar'):
        update_str +=', avatar=:val5'
        update_val[':val5'] = {'N':data['avatar']}
    if data.__contains__('avatar_file_url'):
        update_str +=', avatar_file_url=:val6'
        update_val[':val6'] = {'S':data['avatar_file_url']}
    if data.__contains__('mobile'):
        update_str +=', mobile=:val7'
        update_val[':val7'] = {'S':data['mobile']}
   
    username =  data['user_name']
    table = data['table']
    id =  data['id']
    if id == '' :
        id = username
    # print(update_str) 
    # print(update_val)
    res = client.update_item(
        TableName=table,
        Key={
            'id': {'S':username},
        },
        UpdateExpression = update_str,
        ExpressionAttributeValues=update_val,
        ConditionExpression='attribute_exists(id)' and 'passkey = :pass',
        
    )
    return {
        'statusCode': 200,
        'body': json.dumps('OK')
    }


In [ ]:
#get_item  https://4ejceyu3shifvhawypw3yfvbrq0expbl.lambda-url.us-east-1.on.aws/
import json
import boto3
import base64

client = boto3.client('dynamodb')

def lambda_handler(event, context):
    # TODO implement
    #{"user_name":"yrpo","table":"user","passkey":"chimney123","id":""}
  
    data = json.loads(event['body'])
    table = data['table']
    username = data['user_name']
    passkey = data['passkey']
    id =  data['id']
    if id == '' :
        id = username
    response = client.get_item(
        TableName=table,
        Key={
            'id': {'S': id},
        },
    )
    if response.__contains__('Item'):
         res = response['Item']
         print(res)
    else:
        res = 'NO id'
        return {
            'statusCode': 202,
            'body': res
        }
    if res['passkey']['S'] == passkey:
        return {
            'statusCode': 200,
            'body': res
        }
    else:
        return {
            'statusCode': 202,
            'body': json.dumps('passkey no suc')
        }


In [ ]:
#iot_creat https://dgse52byjk4rtohou37yg4tjpa0uvnrw.lambda-url.us-east-1.on.aws/
import json
import boto3

client = boto3.client('iot')
def lambda_handler(event, context):
    print(event['body'])
    data = json.loads(event['body'])
    name = data['name']
    policyn = 'policy_'+name
    client_id = 'id_'+name
    policy_d ={"Version": "2012-10-17","Statement":[{"Effect": "Allow","Action": "iot:Receive",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topic/lwt/"+name,\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/delta"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Subscribe",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/lwt/"+name,\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/get/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/get/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/delta"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Connect",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:client/"+client_id,\
        "arn:aws:iot:us-east-1:830472830062:client/"+client_id+"_1"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Publish",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update",\
        "arn:aws:iot:us-east-1:830472830062:topic/lwt/"+name
      ]\
    }]
    }
    policy = json.dumps(policy_d)   #先把字典转成json
    #print(name)
    #创建证书和密匙
    res1 = client.create_keys_and_certificate(setAsActive=True)
    #print(res1)
    #创建策略
    response = client.create_policy(
    policyName=policyn,
    policyDocument=policy
    )
    #创建事物
    res3 = client.create_thing(
    thingName=name
    )
    #把策略附加到证书
    res2 = client.attach_policy(
    policyName=policyn,
    target=res1['certificateArn']
    )
    #print(res3)
    #把证书附加到事物
    res4 = client.attach_thing_principal(
    thingName=name,
    principal=res1['certificateArn']
    )
    #创建规则
    # response = client.create_topic_rule(
    #     ruleName=name.replace("-","")+'_rule',
    #     topicRulePayload={"sql": "SELECT * FROM 'my/things/"+name+"/update'",\
    #         "ruleDisabled": False,\
    #         "description": "Turn my/things/ into $aws/things/",\
    #         "actions": [\
    #             {\
    #             "republish": {\
    #             "topic": "$$aws/things/"+name+"/shadow/update",\
    #             "roleArn": "arn:aws:iam::830472830062:role/my-iot-role"\
    #             }\
    #             }\
    #         ]\
           
    #     }
    # )
    res = {'key':res1['keyPair']['PrivateKey'],'pem':res1['certificatePem']}
   # print(res4)
    return {
        'statusCode': 200,
        'body': res
    }